In [17]:
from config import *
from utils import *
import keras
from tensorflow.keras.layers import Embedding, Dense, Dropout, GRU, Bidirectional, Input
from tensorflow.keras.layers import Lambda, Flatten, Concatenate, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
%matplotlib inline
import matplotlib
from matplotlib import pyplot as plt
import gensim
from CapsuleLayers import Capsule
import tensorflow as tf

In [19]:
"""
Load X, y
X = np.load(X_SAVE_PATH)
y = np.load(y_SAVE_PATH)
X_word = X
X_char = [X[i].replace(' ','') for i in range(len(X))]


# Encode labels
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
y = enc.fit_transform(y.reshape((-1,1))).toarray()

# Make char vocabulary
char2vec_f = open(CHAR2VEC_PATH, 'r')
char2vec_txt = char2vec_f.readlines()

# tokenize char
tokenizer_char = Tokenizer(char_level=True)
tokenizer_char.fit_on_texts(X_char)
X_char_seq = tokenizer_char.texts_to_sequences(X_char)
# get char dictionary
char_dictionary = tokenizer_char.word_index

# tokenize word
tokenizer_word = Tokenizer()
tokenizer_word.fit_on_texts(X_word)
X_word_seq = tokenizer_word.texts_to_sequences(X_word)
# get word dictionary
word_dictionary = tokenizer_word.word_index
"""

"\nLoad X, y\nX = np.load(X_SAVE_PATH)\ny = np.load(y_SAVE_PATH)\nX_word = X\nX_char = [X[i].replace(' ','') for i in range(len(X))]\n\n\n# Encode labels\nfrom sklearn.preprocessing import OneHotEncoder\nenc = OneHotEncoder()\ny = enc.fit_transform(y.reshape((-1,1))).toarray()\n\n# Make char vocabulary\nchar2vec_f = open(CHAR2VEC_PATH, 'r')\nchar2vec_txt = char2vec_f.readlines()\n\n# tokenize char\ntokenizer_char = Tokenizer(char_level=True)\ntokenizer_char.fit_on_texts(X_char)\nX_char_seq = tokenizer_char.texts_to_sequences(X_char)\n# get char dictionary\nchar_dictionary = tokenizer_char.word_index\n\n# tokenize word\ntokenizer_word = Tokenizer()\ntokenizer_word.fit_on_texts(X_word)\nX_word_seq = tokenizer_word.texts_to_sequences(X_word)\n# get word dictionary\nword_dictionary = tokenizer_word.word_index\n"

In [24]:
##Load X, y
X = np.load(X_SAVE_PATH)
y = np.load(y_SAVE_PATH)
X_char = [X[i].replace(' ','') for i in range(len(X))]

# create char tokenizer and char embedding_matrix
char2vec_f = open("char_embedding/glove.840B.300d-char.txt",'r')
char2vecs = char2vec_f.readlines()

char_tokenizer = Tokenizer(char_level=True,oov_token=0)
char_embd_matrix = []


for line in char2vecs:
    tokens = line.split(' ')
    char = tokens[0]
    vector =[float(token) for token in tokens[1:]]
    char_tokenizer.fit_on_texts(char)
    char_embd_matrix.append(vector)
char_embd_matrix.insert(0,np.random.uniform(size=len(char_embd_matrix[0])).tolist())
char_embd_matrix = np.asarray(char_embd_matrix)


# create word tokenizer and word embedding matrix
word2vec_f = open("word_embedding/glove.6B.300d.txt",'r')
word_tokenizer = Tokenizer(oov_token=0)
word_embd_matrix = []

count = 0
while(count < 8000):
    line = word2vec_f.readline()
    if not line:
        break
    tokens = line.split(' ')
    word = tokens[0]
    vector = [float(token) for token in tokens[1:]]
    word_tokenizer.fit_on_texts(word)
    word_embd_matrix.append(vector)
    count+=1
word_embd_matrix.insert(0,np.random.uniform(size=len(word_embd_matrix[0])).tolist())
word_embd_matrix = np.asarray(word_embd_matrix)


# convert string to sequence
X_char_seq = char_tokenizer.texts_to_sequences(X_char)
X_word_seq = word_tokenizer.texts_to_sequences(X)

# one hot encode y
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
y = enc.fit_transform(y.reshape((-1,1))).toarray()

In [25]:
"""
# Make char embedding dictionary
char_dict = {}
print('# Making char vocabulary ...')
for i in tqdm(range(len(char2vec_txt))):
    char2vec_i = char2vec_txt[i].split(' ')
    char_dict[char2vec_i[0]] = np.array(char2vec_i[1:], dtype=np.float32)
    
# Make char embedding matrix
print('# Making char embedding matrix ...')
char_embd_matrix = []
for word, i in char_dict.items():
    try:
        vec = char_dict[word]
        char_embd_matrix.append(vec)
    except:
        print('No such char {}'.format(word))
char_embd_matrix = np.array(char_embd_matrix)
char_embd_matrix = np.insert(char_embd_matrix, [0], np.zeros((300)), axis=0)

# Make word embedding matrix
print('# Loading word embedding model ...')
word_dict = gensim.models.KeyedVectors.load_word2vec_format(WORD2VEC_PATH, binary=True)
print('# Making word embedding matrix ...')
word_embd_matrix = []
for word, i in tqdm(word_dictionary.items()):
    try:
        vec = word_dict[word]
        word_embd_matrix.append(vec)
    except:
        print('No such word {}',format(word))
word_embd_matrix = np.array(word_embd_matrix)
word_embd_matrix = np.insert(word_embd_matrix, [0], np.zeros((300)), axis=0)


# Transfer non-meaningful chars to zeros
for i in range(len(X_char_seq)):
    for j in range(len(X_char_seq[i])):
        if X_char_seq[i][j]>len(char_embd_matrix)-1:
            X_char_seq[i][j] = 0

# Transfer non-meaningful words to zeros
for i in range(len(X_word_seq)):
    for j in range(len(X_word_seq[i])):
        if X_word_seq[i][j]>len(word_embd_matrix)-1:
            X_word_seq[i][j] = 0
"""

"\n# Make char embedding dictionary\nchar_dict = {}\nprint('# Making char vocabulary ...')\nfor i in tqdm(range(len(char2vec_txt))):\n    char2vec_i = char2vec_txt[i].split(' ')\n    char_dict[char2vec_i[0]] = np.array(char2vec_i[1:], dtype=np.float32)\n    \n# Make char embedding matrix\nprint('# Making char embedding matrix ...')\nchar_embd_matrix = []\nfor word, i in char_dict.items():\n    try:\n        vec = char_dict[word]\n        char_embd_matrix.append(vec)\n    except:\n        print('No such char {}'.format(word))\nchar_embd_matrix = np.array(char_embd_matrix)\nchar_embd_matrix = np.insert(char_embd_matrix, [0], np.zeros((300)), axis=0)\n\n# Make word embedding matrix\nprint('# Loading word embedding model ...')\nword_dict = gensim.models.KeyedVectors.load_word2vec_format(WORD2VEC_PATH, binary=True)\nprint('# Making word embedding matrix ...')\nword_embd_matrix = []\nfor word, i in tqdm(word_dictionary.items()):\n    try:\n        vec = word_dict[word]\n        word_embd_mat

In [26]:
# Padding seqences 
print('# Padding ...')
max_len_char = max([len(X_char_seq[i]) for i in range(len(X_char_seq))])
X_char_seq = pad_sequences(sequences=X_char_seq, maxlen=max_len_char)

max_len_word = max([len(X_word_seq[i]) for i in range(len(X_word_seq))])
X_word_seq = pad_sequences(sequences=X_word_seq, maxlen=max_len_word)

# Padding ...


In [27]:
# Load images files names
train_imgs_fn = []
for roots, dirs, files in os.walk(TRAIN_IMG_PATH):
    train_imgs_fn = files

# Reshape images to (150, 500)
print('# Loading images ...')
X_train_imgs = []
for i in tqdm(range(len(train_imgs_fn))):
    img_ = cv2.imread(os.path.join(TRAIN_IMG_PATH, train_imgs_fn[i]))
    X_train_imgs.append(cv2.resize(img_, (300,150)))
X_train_imgs = np.array(X_train_imgs)
# X_train_imgs = np.mean(X_train_imgs, axis=3)
# X_train_imgs = np.expand_dims(X_train_imgs, axis=3)

  8%|▊         | 268/3193 [00:00<00:01, 2675.21it/s]

# Loading images ...


100%|██████████| 3193/3193 [00:00<00:00, 3320.72it/s]


In [28]:
# Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
print('# Building model ...')

# # InceptionV3 module
# inx = InceptionV3(include_top=False, weights='imagenet', input_shape=(150, 300, 3))
# for layer in inx.layers:
#     layer.trainable=False
# input_inx = inx.input
# inx = inx.get_layer('activation_3').output
# inx = MaxPooling2D(2)(inx)
# inx = Flatten()(inx)

# RNN char module
input_char = Input(shape=(max_len_char, ))
char_embd = Embedding(input_dim=len(char_embd_matrix),
                 output_dim=300,
                weights=[char_embd_matrix],
                mask_zero=True)(input_char)

# RNN word module
input_word = Input(shape=(max_len_word,))
word_embd = Embedding(input_dim=len(word_embd_matrix),
                 output_dim=300,
                weights=[word_embd_matrix],
                mask_zero=True)(input_word)

outputs = []
for i in range(7):
    char = Bidirectional(GRU(32))(char_embd)
    char = Lambda(lambda x:tf.expand_dims(x, axis=1))(char)
    char = Capsule(5, 8, 5)(char)
    char = Flatten()(char)
    
    word = Bidirectional(GRU(32))(word_embd)
    word = Lambda(lambda x:tf.expand_dims(x, axis=1))(word)
    word = Capsule(5, 8, 5)(word)
    word = Flatten()(word)
    
    out = Concatenate()([char, word])
    out = Dense(32)(out)
    out = Dense(16)(out)
    out = Dropout(0.3)(out)
    out = Dense(1, activation='sigmoid')(out)
    outputs.append(out)
model = Model(inputs=[input_char, input_word], outputs=outputs)
# model = Model(inputs=input_inx, outputs=outputs)
# Compile
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])
model.summary()

# Building model ...
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 261)          0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 79)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 261, 300)     28500       input_2[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 79, 300)      2400300     input_3[0][0]                    
________________________________________________________________________________________

In [29]:
# Split data
print('# Spliting data ...')
from sklearn.model_selection import train_test_split
# X_img_train, X_img_valid,  y_train, y_valid = train_test_split(X_train_imgs,y, test_size=0.2)
X_char_train, X_char_valid, X_word_train, X_word_valid, y_train, y_valid = train_test_split(X_char_seq, 
                                                                                            X_word_seq,
                                                                                            y, test_size=0.2)

# Spliting data ...


In [30]:
# Train
print('# Training ...')
# model.fit(x=X_img_train, y=[y_train[:,i] for i in range(7)], validation_data=[X_img_valid, y_valid], batch_size=64, epochs=30)
model.fit(x=[X_char_train, X_word_train], y=[y_train[:,i] for i in range(7)], 
          validation_data=[[X_char_valid, X_word_valid], [y_valid[:,i] for i in range(7)]], batch_size=64, epochs=30)

# Training ...


/home/donald/anaconda3/envs/tf36/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 2554 samples, validate on 639 samples
Epoch 1/30
2554/2554 [==============================] - 217s 85ms/step - loss: 2.7074 - dense_2_loss: 0.2558 - dense_5_loss: 0.4662 - dense_8_loss: 0.2292 - dense_11_loss: 0.4830 - dense_14_loss: 0.5624 - dense_17_loss: 0.3285 - dense_20_loss: 0.3821 - dense_2_binary_accuracy: 0.9667 - dense_5_binary_accuracy: 0.8567 - dense_8_binary_accuracy: 0.9804 - dense_11_binary_accuracy: 0.8610 - dense_14_binary_accuracy: 0.7866 - dense_17_binary_accuracy: 0.8645 - dense_20_binary_accuracy: 0.8132 - val_loss: 1.9834 - val_dense_2_loss: 0.2140 - val_dense_5_loss: 0.3288 - val_dense_8_loss: 0.1678 - val_dense_11_loss: 0.2970 - val_dense_14_loss: 0.4337 - val_dense_17_loss: 0.2768 - val_dense_20_loss: 0.2653 - val_dense_2_binary_accuracy: 0.9593 - val_dense_5_binary_accuracy: 0.9296 - val_dense_8_binary_accuracy: 0.9922 - val_dense_11_binary_accuracy: 0.9421 - val_dense_14_binary_accuracy: 0.8059 - val_dense_17_binary_accuracy: 0.8873 - val_dense_20_bi

KeyboardInterrupt: 

In [31]:
model.save_weights('./clf.h5')

In [167]:
def retrieveBbox(img_n, box_n):
    bbox = np.load(os.path.join(test_split_img_dir, img_n, 'bbox.npy'))
    return bbox.item()[box_n]

In [168]:
test_split_img_dir = '/home/donald/PycharmProjects/Atos/data/test_split_img'
test_rotated_img_dir = '/home/donald/PycharmProjects/Atos/data/test_rotated_img_box/'
test_string_data_dir = '/home/donald/Blaine/string_data'

In [199]:
def predictBbox(string_data_dir, n, img_dir=test_split_img_dir, save_name='test.jpg'):
    # Read stings & string_belong
    str_f = open(os.path.join(string_data_dir,'OCRed_string.txt'))
    belong_f = open(os.path.join(string_data_dir, 'string_belong.txt'))

    test_str = str_f.readlines()
    test_str_belong = belong_f.readlines()
    test_str_belong = [test_str_belong[i].replace('\n', '').split('\t') for i in range(len(test_str_belong))]

    # Generate Input Data
    test_char = [test_str[i].replace('\n','').replace(' ','') for i in range(len(test_str))]
    test_word = [test_str[i].replace('\n','') for i in range(len(test_str_belong))]

    test_char = tokenizer_char.texts_to_sequences(test_char)
    test_word = tokenizer_word.texts_to_sequences(test_word)

    test_char = pad_sequences(sequences=test_char, maxlen=max_len_char)
    test_word = pad_sequences(sequences=test_word, maxlen=max_len_word)
    
    # Predict
    pred = model.predict(x=[test_char, test_word])
    
    # Top n indices
    inds = [np.argpartition(np.reshape(a=pred[i], newshape=(len(pred[i]))), -n)[-n:]
           for i in range(7)]
    
    # Resulting Boxes
    results = []
    for i in range(len(inds)):
        ind = inds[i]
        boxes = []
        for ind_ in ind:
            boxes.append(retrieveBbox(test_str_belong[ind_][0][:-4], test_str_belong[ind_][1]))
        result = np.mean(boxes, axis=0)
        results.append(result)
    results = np.array(results, dtype=int)

    # Plot 
#     img = cv2.imread(os.path.join(img_dir, test_str_belong[0][0]))
#     img_ = img.copy()
#     for i in range(len(results)):
#         img_ = cv2.rectangle(img, tuple(results[i][0]), tuple(results[i][2]), color=(0,255,0), thickness=2)
#     plt.imshow(img_)
    
    # Save
#     cv2.imwrite('./testing/{}.jpg'.format(save_name), img_)
    
    return results

In [200]:
test_string_dir = '/home/donald/Blaine/SPLIT_OUTPUT'

In [201]:
images_n = []
for roots, dirs, files in os.walk(test_string_dir):
    images_n.extend(dirs)

In [202]:
results = {}
for i in tqdm(range(len(images_n))):
    results[images_n[i]] = predictBbox(os.path.join(test_string_dir, images_n[i]), 2, save_name=images_n[i])



  0%|          | 0/302 [00:00<?, ?it/s]

  0%|          | 1/302 [00:19<1:36:11, 19.18s/it]

  1%|          | 2/302 [00:40<1:41:25, 20.29s/it]

  1%|          | 3/302 [01:02<1:43:05, 20.69s/it]

  1%|▏         | 4/302 [01:16<1:34:53, 19.11s/it]

  2%|▏         | 5/302 [01:29<1:28:39, 17.91s/it]

  2%|▏         | 6/302 [02:00<1:39:05, 20.09s/it]

  2%|▏         | 7/302 [02:37<1:50:34, 22.49s/it]

  3%|▎         | 8/302 [03:38<2:13:36, 27.27s/it]

  3%|▎         | 9/302 [04:06<2:13:50, 27.41s/it]

  3%|▎         | 10/302 [04:19<2:06:27, 25.98s/it]

  4%|▎         | 11/302 [04:26<1:57:42, 24.27s/it]

  4%|▍         | 12/302 [04:50<1:57:08, 24.24s/it]

  4%|▍         | 13/302 [05:18<1:57:55, 24.48s/it]

  5%|▍         | 14/302 [05:37<1:55:39, 24.10s/it]

  5%|▍         | 15/302 [06:20<2:01:16, 25.35s/it]

  5%|▌         | 16/302 [06:26<1:55:04, 24.14s/it]

  6%|▌         | 17/302 [06:48<1:54:00, 24.00s/it]

  6%|▌         | 18/302 [07:15<1:54:29, 24.19s/it]

  6%|▋         | 19/302 [08:00

 46%|████▋     | 140/302 [50:31<58:27, 21.65s/it]

 47%|████▋     | 141/302 [50:45<57:57, 21.60s/it]

 47%|████▋     | 142/302 [51:04<57:33, 21.58s/it]

 47%|████▋     | 143/302 [51:38<57:25, 21.67s/it]

 48%|████▊     | 144/302 [51:59<57:03, 21.67s/it]

 48%|████▊     | 145/302 [52:28<56:49, 21.71s/it]

 48%|████▊     | 146/302 [52:35<56:12, 21.62s/it]

 49%|████▊     | 147/302 [52:57<55:50, 21.61s/it]

 49%|████▉     | 148/302 [53:12<55:22, 21.57s/it]

 49%|████▉     | 149/302 [53:45<55:11, 21.65s/it]

 50%|████▉     | 150/302 [54:30<55:14, 21.80s/it]

 50%|█████     | 151/302 [55:01<55:01, 21.87s/it]

 50%|█████     | 152/302 [55:24<54:40, 21.87s/it]

 51%|█████     | 153/302 [55:49<54:22, 21.89s/it]

 51%|█████     | 154/302 [56:34<54:21, 22.04s/it]

 51%|█████▏    | 155/302 [56:40<53:44, 21.94s/it]

 52%|█████▏    | 156/302 [56:47<53:08, 21.84s/it]

 52%|█████▏    | 157/302 [57:11<52:49, 21.86s/it]

 52%|█████▏    | 158/302 [57:48<52:40, 21.95s/it]

 53%|█████▎    | 159/302 [58:00

 92%|█████████▏| 279/302 [1:42:18<08:26, 22.00s/it]

 93%|█████████▎| 280/302 [1:42:25<08:02, 21.95s/it]

 93%|█████████▎| 281/302 [1:42:45<07:40, 21.94s/it]

 93%|█████████▎| 282/302 [1:43:11<07:19, 21.96s/it]

 94%|█████████▎| 283/302 [1:43:21<06:56, 21.91s/it]

 94%|█████████▍| 284/302 [1:44:02<06:35, 21.98s/it]

 94%|█████████▍| 285/302 [1:44:11<06:12, 21.94s/it]

 95%|█████████▍| 286/302 [1:44:51<05:51, 22.00s/it]

 95%|█████████▌| 287/302 [1:45:17<05:30, 22.01s/it]

 95%|█████████▌| 288/302 [1:45:51<05:08, 22.06s/it]

 96%|█████████▌| 289/302 [1:46:27<04:47, 22.10s/it]

 96%|█████████▌| 290/302 [1:46:38<04:24, 22.06s/it]

 96%|█████████▋| 291/302 [1:47:02<04:02, 22.07s/it]

 97%|█████████▋| 292/302 [1:47:19<03:40, 22.05s/it]

 97%|█████████▋| 293/302 [1:47:53<03:18, 22.09s/it]

 97%|█████████▋| 294/302 [1:48:35<02:57, 22.16s/it]

 98%|█████████▊| 295/302 [1:48:44<02:34, 22.12s/it]

 98%|█████████▊| 296/302 [1:49:04<02:12, 22.11s/it]

 98%|█████████▊| 297/302 [1:49:18<01:50, 22.08

In [197]:
import pandas as pd

In [198]:
df = pd.DataFrame(columns=['image_name','found_1','X1_1','Y1_1','X2_1','Y2_1','X3_1','Y3_1','X4_1','Y4_1',
                          'found_2','X1_2','Y1_2','X2_2','Y2_2','X3_2','Y3_2','X4_2','Y4_2',
                          'found_3','X1_3','Y1_3','X2_3','Y2_3','X3_3','Y3_3','X4_3','Y4_3',
                          'found_4','X1_4','Y1_4','X2_4','Y2_4','X3_4','Y3_4','X4_4','Y4_4',
                          'found_5','X1_5','Y1_5','X2_5','Y2_5','X3_5','Y3_5','X4_5','Y4_5',
                          'found_6','X1_6','Y1_6','X2_6','Y2_6','X3_6','Y3_6','X4_6','Y4_6',
                          'found_7','X1_7','Y1_7','X2_7','Y2_7','X3_7','Y3_7','X4_7','Y4_7'])